In [ ]:
# This is time taking unfortunately

In [2]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
from tqdm import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

import jsonlines 
from tqdm import tqdm
from bs4 import BeautifulSoup

import exrex
from string import punctuation

In [3]:
Diseases = '/nfs/gns/literature/lit-textmining-pipelines/automata/DiseaseDictionary.mwt' #(# of terms - 59088)
Genes_Proteins = '/nfs/gns/literature/lit-textmining-pipelines/automata/swissprot_Sept2014.2.3.mwt' #(# of terms - 347929)
Organism = '/nfs/gns/literature/lit-textmining-pipelines/automata/Organisms150507.2.mwt' #(# of terms - 1683021)

gene_patterns = []
disease_patterns = []
organisms_patterns = []


# print('Loading Genes Dictionary')
# with open(Genes_Proteins, 'r') as f:
#     contents = f.read()
#     soup = BeautifulSoup(contents, 'lxml')
#     pattern_types = soup.find_all('t')
#     for pattern in tqdm(pattern_types):
#         gene_patterns.append(pattern.text)
#     pattern_types = soup.find_all('r')
#     for pattern in tqdm(pattern_types):
#         gene_patterns.append(pattern.text)    
        
        

# print('loading Disease Dictionary')
# with open(Diseases, 'r') as f:
#     contents = f.read()
#     soup = BeautifulSoup(contents, 'lxml')
#     pattern_types = soup.find_all('t')
#     for pattern in tqdm(pattern_types):
#         disease_patterns.append(pattern.text)


        
# Organism = '/nfs/gns/literature/machine-learning/Dictionaries/MWT/organisms_toy.mwt' #(# of terms - 1683021)        
# organisms_patterns = []
print('Loading Organims Dictionary')
with open(Organism, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    pattern_types = soup.find_all('r')
for pattern in tqdm(pattern_types):
    dictionary_term = exrex.getone(pattern.text.replace('[ \\-_]*', ' ').replace('[^A-Za-z0-9]', ''))
    clean_term = ' '.join(dictionary_term.split())
    organisms_patterns.append(clean_term)
  

Loading Organims Dictionary


100%|██████████| 1683023/1683023 [02:01<00:00, 13859.43it/s]


In [ ]:
best_model_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best'
nlp = spacy.load(best_model_path)
print("Loaded from", best_model_path)

In [4]:
nlp = spacy.blank('en')

In [5]:
from spacy_lookup import Entity

# dict_GP = Entity(keywords_list=gene_patterns, label='GP', case_sensitive=False)
# dict_DS = Entity(keywords_list=disease_patterns, label='DS')
dict_OG = Entity(keywords_list=organisms_patterns, label='OG')



In [ ]:
gene_patterns

In [6]:
# dict_GP.name ='GP_dict'
# dict_DS.name ='DS_dict'
dict_OG.name = 'OG_dict'

In [13]:
# nlp.add_pipe(dict_GP, last=True)
nlp.add_pipe(dict_OG, last=True)
# nlp.add_pipe(dict_DS, last=True)

In [14]:
nlp.pipeline

[('OG_dict', <spacy_lookup.Entity at 0x2aef004a53c8>)]

In [15]:
doc = nlp(u"Hereditary breast cancer is brca1 known for its strong tendency of inheritance. Most hereditary breast cancers are related to BRCA1/BRCA2 pathogenic variants. The lifelong risk of breast cancer in pathogenic BRCA1 and BRCA2 variant carriers is approximately 65% and 45%, respectively, whereas that of ovarian cancer is estimated to be 39% and 11%, respectively. Therefore, understanding these variants and clinical knowledge on their occurrence in breast cancers and carriers are important. BRCA1 pathogenic variant breast cancer shows more aggressive clinicopathological features than the BRCA2 pathogenic variant breast cancer. Compared with sporadic breast cancer, their prognosis is still debated. Treatments of BRCA1/BRCA2 pathogenic variant breast cancer are similar to those for BRCA-negative breast cancer, mainly including surgery, radiotherapy, and chemotherapy. Recently, various clinical trials have investigated poly (adenosine diphosphate [ADP]-ribose) polymerase (PARP) inhibitor treatment for advanced-stage BRCA1/BRCA2 pathogenic variant breast cancer. Among the various PARP inhibitors, olaparib and talazoparib, which reached phase III clinical trials, showed improvement of median progression-free survival around three months. Preventive and surveillance in human and mouse strategies for BRCA pathogenic variant breast cancer to reduce cancer recurrence and improve treatment outcomes have recently received increasing attention. In this review, we provide an information on the clinical features of BRCA1/BRCA2 pathogenic variant breast cancer and clinical recommendations for BRCA pathogenic variant carriers, with a focus on treatment and prevention strategies. With this knowledge, clinicians could manage the BRCA1/BRCA2 pathogenic variant breast cancer patients more effectively.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

cancers 103 110 OG
cancers 455 462 OG
human 1279 1284 OG
this 1455 1459 OG
this 1692 1696 OG


In [ ]:
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import csv


gene_patterns_ = set(gene_patterns)
disease_patterns_ = set(disease_patterns)
organisms_patterns_ = set(organisms_patterns)

train_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/train.csv'
dev_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/dev.csv'

train_df = pd.read_csv(train_set, sep = '\t', names = ['pmcid', 'sentence','ner'])
dev_df = pd.read_csv(dev_set, sep = '\t', names = ['pmcid', 'sentence','ner'])

for index, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    ner_tags = row['ner']
    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
#             print(ner_tags)
        except:
            pass
    try:        
        for each_tag in ner_tags:
            token_ = each_tag[2].lower()
            ner_ = each_tag[3]

            if ner_ == 'GP':
                gene_patterns_.add(token_)
            elif ner_ == 'OG':
                organisms_patterns_.add(token_)
            elif ner_ == 'DS':
                disease_patterns_.add(token_)  
    except:
        pass

            
for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
    ner_tags = row['ner']
    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
#             print(ner_tags)
        except:
            pass
    try:        
        for each_tag in ner_tags:
            token_ = each_tag[2].lower()
            ner_ = each_tag[3]

            if ner_ == 'GP':
                gene_patterns_.add(token_)
            elif ner_ == 'OG':
                organisms_patterns_.add(token_)
            elif ner_ == 'DS':
                disease_patterns_.add(token_)  
    except:
        pass               

In [ ]:
## load the best model

best_model_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best'

nlp = spacy.load(best_model_path)
print("Loaded from", best_model_path)


In [8]:
# nlp.add_pipe(entity_matcher_OG, after='GP_dict')
print(nlp.pipe_names)

[]


In [16]:
text = 'Interspecific tumor PT overgrowth phenocopies the s.Crocodylurus tarahumaras frogs female gametophytic mutants feronia/sirène (fer/srn), lorelei (lre), nortia (nta), TuRan (tun), evAn (evn), and Zea mays embryo sac 4 (ZmES4) RNAi-lines1314151617181920, which are defective in the reception of intraspecific PTs. '
# text = 'These results indicate the dentate gyrus is mostly comprised of mature neurons (NeuN), along with a smaller population of precursor cells (nestin) and newly differentiated neurons (DCX), which corresponds to prior findings examining the relative number of each cell population in the dentate gyrus, indicating the proportion of cells labeled by the sensor approximately reflects physiological proportions [21, 22]. '
sentence =nlp(text)

print(sentence)
for ent in sentence.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Interspecific tumor PT overgrowth phenocopies the s.Crocodylurus tarahumaras frogs female gametophytic mutants feronia/sirène (fer/srn), lorelei (lre), nortia (nta), TuRan (tun), evAn (evn), and Zea mays embryo sac 4 (ZmES4) RNAi-lines1314151617181920, which are defective in the reception of intraspecific PTs. 
frogs 77 82 OG
nortia 152 158 OG
Zea mays 195 203 OG


In [ ]:
nlp.to_disk("/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/dictionary_en-pubmed-pmc-lg/")

In [ ]:
gene_patterns

In [10]:
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize


def convert2IOB(text_data, ner_tags):
    tokenizer = WordPunctTokenizer()

    tokens = []
    ners = []
    spans = []

    split_text = tokenizer.tokenize(text_data)
    span_text = list(tokenizer.span_tokenize(text_data))
    # for each word token append 'O'
    arr = ['O'] * len(split_text)

    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
        except:
            pass

    elif isinstance(ner_tags, float) or ner_tags is None:
        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

    for each_tag in ner_tags:
        span_list = (each_tag[0], each_tag[1])
        token_list = wordpunct_tokenize(each_tag[2])
        ner_list = wordpunct_tokenize(each_tag[3])

        if (len(token_list) > len(ner_list)):
            ner_list = len(token_list) * ner_list
        for i in range(0, len(ner_list)):
            # The logic here is look for the first B-tag and then append I-tag next
            if (i == 0):
                ner_list[i] = 'B-' + ner_list[i]
            else:
                ner_list[i] = 'I-' + ner_list[i]

        tokens.append(token_list)
        ners.append(ner_list)
        spans.append(span_list)

    split_token_span_list = list(zip(split_text, span_text))
    span_ner_list = list(zip(spans, ners))

    sub_spans = []  # get sub spans from the full spans of the ner

    for each_span_ner_list in span_ner_list:
        # in full range ner e.g., [144, 150, 'GM-CSF', 'GP']
        count = 0
        # count is to keep track of the B, I, sub tags in the ner list
        for each_token in split_token_span_list:
            sub_spans_ = find_sub_span(each_token[1], each_span_ner_list[0])
            if sub_spans_:
                sub_spans.append([sub_spans_, each_span_ner_list[1][count]])
                count = count + 1

    for i, each_span_token in enumerate(split_token_span_list):
        for each_ner_span in sub_spans:
            if each_span_token[1] == each_ner_span[0]:
                arr[i] = ''.join(each_ner_span[1])

    return zip(split_text, arr)


In [17]:
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import csv


def find_sub_span(sub_span_range, full_spans_range):
    # if a sub span is present in full span return it
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range
    
    

test_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/test.csv'

result_path = '/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/Dictionary_fused_en_pubmed_pmc_lg/'


df_45 = pd.read_csv(test_set, sep = '\t', names = ['pmcid', 'sentence','ner'])

with open(result_path + 'Dictionary_OG_iob.csv', 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index, row in tqdm(df_45.iterrows(), total=df_45.shape[0]):
#         print(row['ner'])
        text = row['sentence'] # .encode('utf-8').decode('utf-8')
#         print(text)
        sentence = nlp(text)
        ml_ner =[]
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)

        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')


100%|██████████| 17957/17957 [00:57<00:00, 313.66it/s]


In [ ]:
## Test best model performance on 2000 set

from ast import literal_eval

epmc_annotations_2000 = '/nfs/gns/literature/machine-learning/evaluation/2000articles/europePMC-NER/annotations_API/full_sentences/tagged_sentences/Europe_PMC_annotation.csv'

result_path = '/nfs/gns/literature/machine-learning/evaluation/2000articles/ML-NER/Dictionary_fused_en_pubmed_pmc_lg/'


df_2000 = pd.read_csv(epmc_annotations_2000, sep = '\t', names = ['pmcid', 'section', 'sentence','ner'])


with open(result_path + 'Dictionary_and_trainingset_en-pubmed-pmc-lg_2000_iob.csv', 'a', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index, row in tqdm(df_2000.iterrows(), total=df_2000.shape[0]):
#         print(row['ner'])
        text = row['sentence'].encode('utf-8').decode('utf-8')
#         print(text)
        sentence = nlp(text)
        ml_ner =[]
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)

        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
